In [22]:
!pip install plotly

In [23]:
import numpy as np
import torch
import sklearn
import plotly
import pandas as pd

## Understanding the dataset!

In [24]:
data = pd.read_csv("./U.S._Chronic_Disease_Indicators__CDI_.csv")
data.head()

C:\Users\Windows\AppData\Local\Temp\ipykernel_25568\1647966677.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("./U.S._Chronic_Disease_Indicators__CDI_.csv")


,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,...,LocationID,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1,StratificationCategoryID2,StratificationID2,StratificationCategoryID3,StratificationID3
0,2014,2014,AR,Arkansas,SEDD; SID,Asthma,Hospitalizations for asthma,NaN,NaN,Number,...,5,AST,AST3_1,NMBR,GENDER,GENM,NaN,NaN,NaN,NaN
1,2018,2018,CO,Colorado,SEDD; SID,Asthma,Hospitalizations for asthma,NaN,NaN,Number,...,8,AST,AST3_1,NMBR,OVERALL,OVR,NaN,NaN,NaN,NaN
2,2018,2018,DC,District of Columbia,SEDD; SID,Asthma,Hospitalizations for asthma,NaN,NaN,Number,...,11,AST,AST3_1,NMBR,OVERALL,OVR,NaN,NaN,NaN,NaN
3,2017,2017,GA,Georgia,SEDD; SID,Asthma,Hospitalizations for asthma,NaN,NaN,Number,...,13,AST,AST3_1,NMBR,GENDER,GENF,NaN,NaN,NaN,NaN
4,2010,2010,MI,Michigan,SEDD; SID,Asthma,Hospitalizations for asthma,NaN,NaN,Number,...,26,AST,AST3_1,NMBR,RACE,HIS,NaN,NaN,NaN,NaN


In [25]:
# I remove these columns because all of the entries are null and thus not useful. I can't extrapolate any data from this and thus, not neccessary for analysis.
data.drop(columns=['Response', 
                   'StratificationCategory2', 
                   'StratificationCategory3', 
                   'Stratification3', 
                   'ResponseID', 
                   'StratificationCategoryID2', 
                   'StratificationID2', 
                   'StratificationCategoryID3', 
                   'StratificationID3'], inplace=True)
data.columns

Index(['YearStart', 'YearEnd', 'LocationAbbr', 'LocationDesc', 'DataSource',
       'Topic', 'Question', 'DataValueUnit', 'DataValueType', 'DataValue',
       'DataValueAlt', 'DataValueFootnoteSymbol', 'DatavalueFootnote',
       'LowConfidenceLimit', 'HighConfidenceLimit', 'StratificationCategory1',
       'Stratification1', 'Stratification2', 'GeoLocation', 'LocationID',
       'TopicID', 'QuestionID', 'DataValueTypeID', 'StratificationCategoryID1',
       'StratificationID1'],
      dtype='object')

In [26]:
# Understand the data types associated with each column of the data.
# Also parse this information for plotting for visual context of data!
import os

regression_columns = []                     # Generate a list of columns that have numerical values. We can use regression techniques on this data.
classification_columns = []                 # Generate a list of columns that have categorical values. We can use classification techniques on this data.
with open("./output.txt", "wb") as file:       # Output this to a file so that 
    for col, dtype in zip(data, data.dtypes):
        print(f"{col}\t\t{dtype}")
        if dtype == object:
            print(data[col].unique())
        else:
            print(data[col].describe())
        print()

YearStart		int64
count    1.185676e+06
mean     2.015103e+03
std      3.320259e+00
min      2.001000e+03
25%      2.013000e+03
50%      2.015000e+03
75%      2.018000e+03
max      2.021000e+03
Name: YearStart, dtype: float64

YearEnd		int64
count    1.185676e+06
mean     2.015643e+03
std      3.001197e+00
min      2.001000e+03
25%      2.013000e+03
50%      2.016000e+03
75%      2.018000e+03
max      2.021000e+03
Name: YearEnd, dtype: float64

LocationAbbr		object
['AR' 'CO' 'DC' 'GA' 'MI' 'MT' 'OR' 'PR' 'WI' 'AL' 'ID' 'IL' 'KS' 'LA'
 'MA' 'MD' 'MN' 'MS' 'NC' 'NM' 'TX' 'NY' 'IN' 'NV' 'SC' 'VA' 'IA' 'UT'
 'WY' 'AK' 'CA' 'OH' 'US' 'HI' 'WA' 'SD' 'DE' 'KY' 'ND' 'RI' 'VI' 'VT'
 'AZ' 'FL' 'NE' 'OK' 'GU' 'NJ' 'MO' 'ME' 'CT' 'NH' 'TN' 'PA' 'WV']

LocationDesc		object
['Arkansas' 'Colorado' 'District of Columbia' 'Georgia' 'Michigan'
 'Montana' 'Oregon' 'Puerto Rico' 'Wisconsin' 'Alabama' 'Idaho' 'Illinois'
 'Kansas' 'Louisiana' 'Massachusetts' 'Maryland' 'Minnesota' 'Mississippi'
 'North Caro

## Split by Topic